In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/home/pranjal/Downloads/custom_neural_network_library/housing_train.csv")

In [3]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

df = df_column_switch(df, "longitude", "median_house_value")

In [4]:
data = np.array(df)
m,n = data.shape
np.random.shuffle(data)
print(f"m:{m}")
print(f"n:{n}")

m:17000
n:9


In [5]:
val_data = data[0 : 3000].T
Y_val = val_data[0]
X_val = val_data[1 : n]
X_val = X_val / 255.

train_data = data[3000 : m].T
Y_train = train_data[0]
X_train = train_data[1 : n]
X_train = X_train / 255.

In [6]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(8, 14000)
(8, 3000)
(14000,)
(3000,)


In [7]:
def parameterss(hidden_layers, n_neurons, n_classes):
        weights = {}
        biases = {}

        for i in range(hidden_layers):

            if i == 0:
                #For the first hidden layer
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, 270) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5
            else:
                # For rest of the hidden layers
                weights[f"W{i + 1}"] = np.random.rand(n_neurons, n_neurons) - 0.5
                biases[f"b{i + 1}"] = np.random.rand(n_neurons, 1) - 0.5

        #For final output layer
        weights[f"W{hidden_layers + 1}"] = np.random.rand(n_classes, n_neurons) - 0.5
        biases[f"b{hidden_layers + 1}"] = np.random.rand(n_classes, 1) - 0.5

        return weights, biases

In [8]:
w, b = parameterss(2,300,1)

In [9]:
w.keys()

dict_keys(['W1', 'W2', 'W3'])

In [10]:
b.keys()

dict_keys(['b1', 'b2', 'b3'])

In [11]:
w['W1']

array([[ 0.44225761,  0.10639289, -0.43178469, ...,  0.28422916,
        -0.17269672,  0.29221702],
       [-0.29086162,  0.48424328, -0.09606356, ...,  0.27883471,
         0.31524306, -0.25119692],
       [ 0.48185101,  0.31397271,  0.06200468, ..., -0.07325756,
         0.26107497,  0.47844716],
       ...,
       [-0.30767001, -0.36426386,  0.16724888, ...,  0.10324997,
         0.2149969 ,  0.11141046],
       [-0.17203174,  0.21825914,  0.02343457, ..., -0.35585339,
        -0.38930771,  0.36345267],
       [-0.07364417,  0.29866869, -0.45312715, ...,  0.03330314,
        -0.29905392,  0.04712626]])

In [12]:
def ReLU(Z):
    return np.maximum(Z ,0)
    
def Linear(Z):
    return Z   

def derivative_ReLU(Z):
    return Z > 0

def derivative_linear(Z):
    return 1

In [13]:
def forward_propagation_regression(weights, biases, X):

        l = len(weights)
        p = {}

        for i in range(1, l+1):

            if i == 1 :

                p[f"Z{i}"] = weights['W1'].dot(X) + biases['b1']
                p[f"A{i}"] = ReLU( weights['W1'].dot(X) + biases['b1'])

            elif i > 1 and i < l:

                p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
                p[f"A{i}"] = ReLU(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])

            elif i == l:
                
                p[f"Z{i}"] = weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"]
                p[f"A{i}"] = Linear(weights[f"W{i}"].dot(p[f"A{i-1}"]) + biases[f"b{i}"])


        return p

In [14]:
a = forward_propagation(w, b, X_train)

ValueError: shapes (300,270) and (8,14000) not aligned: 270 (dim 1) != 8 (dim 0)

In [ ]:
a.keys()

In [ ]:
def back_propagation(a, weights, X_train, Y_train):
    p = {}
    l = len(weights)

    for i in range(l, -1+1, -1):
        if i == l:
            p[f"dZ{i}"] = a[f"A{i}"] - Y_train
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i < l and i > 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_ReLU(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(a[f"A{i-1}"].T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)
        elif i == 1:
            p[f"dZ{i}"] = weights[f"W{i+1}"].T.dot(p[f"dZ{i+1}"]) * derivative_linear(a[f"Z{i}"])
            p[f"dW{i}"] = 1 / m * p[f"dZ{i}"].dot(X_train.T)
            p[f"db{i}"] = 1 / m * np.sum(p[f"dZ{i}"], axis=1, keepdims=True)

    return  p

In [ ]:
s = back_propagation(a, w, X_train, Y_train)

In [ ]:
# l = 3
# for i in range(l,-1+1,-1):
#     print(i)

In [ ]:
s.keys()

In [ ]:
def update_parameters(weights, biases, s, alpha):
    p1 = {}
    p2 = {}

    l = len(weights)

    for i in range(1, l + 1):
        p1[f"W{i}"] = weights[f"W{i}"] - alpha * s[f"dW{i}"]
        p2[f"b{i}"] = biases[f"b{i}"] - alpha * s[f"db{i}"]

    return p1, p2

In [ ]:
w, b = update_parameters(w, b, s, 0.1)

In [ ]:
w.keys()

In [ ]:
b.keys()

In [ ]:
w['W1']

In [ ]:
print(a['A3'])
print(a['A3'].shape)

In [ ]:
print(Y_train)
print(Y_train.shape)

In [ ]:
def mean_squared_error(Y_true, Y_pred):
    return np.mean((Y_true - Y_pred)**2)

In [ ]:
last_element_key, last_element_value = list(a.items())[-1]
print(last_element_key)

In [ ]:
prediction = mean_squared_error(last_element_value, Y_train)

In [ ]:
prediction